<a href="https://colab.research.google.com/github/abhinavbammidi1401/LLM/blob/main/2348509_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install the libraries

In [1]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb (Optional)(Use Weights and Biases API to build better models faster)
! pip install transformers[torch]
! pip install accelerate -U  #library to help users easily train a Transformers model on any type of distributed setup, whether it is multiple GPU’s on one machine or multiple GPU’s across several machines.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [2]:
!pip install wandb
# log in to WandB
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
   import wandb
   wandb.login()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.5 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#Sets up the environment to work with transformers, datasets, and other essential libraries to perform sequence-to-sequence language modeling tasks using the Hugging Face library.

In [3]:
import torch
import numpy as np
import datasets
from transformers import (
   AutoModelForSeq2SeqLM,
   AutoTokenizer,
   Seq2SeqTrainingArguments,
   Seq2SeqTrainer,
   DataCollatorForSeq2Seq,
)
from tabulate import tabulate
import nltk
from datetime import datetime


#Load the pretrained BART-based model for sequence-to-sequence language modeling, sets the appropriate tokenizer, and defines the maximum lengths for the encoder and decoder sequences.

In [4]:
#model_name to indicate the specific pretrained model used
pretrained_model_name = "sshleifer/distilbart-xsum-12-3"
# Load the pretrained model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
# Set model parameters or use the default
# print(model.config)
# Tokenization parameters
encoder_max_len = 256
decoder_max_len = 64



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

#Importing and Pre-processing

In [6]:
# Load dataset from the CS
import pandas as pd
geographical_data_df = pd.read_csv('/content/Geographicaldata.csv', encoding='latin-1', sep=',',on_bad_lines='skip') #The encoding='latin-1' argument is used to handle invalid characters appropriately.
# Reduce the dataset to 200 rows
geographical_data_df = geographical_data_df.head(200)


In [7]:
print(geographical_data_df.columns)

Index(['City Description', 'Feature Extraction'], dtype='object')


#Splits the dataset into training and validation sets
#Convert pandas DataFrame to a Dataset object using the Hugging Face datasets library.

#Splits it into training and validation sets for further processing and modeling tasks.

In [8]:
import pandas as pd
from datasets import Dataset
# Convert DataFrame to Dataset
geographical_data = Dataset.from_pandas(geographical_data_df)
def flatten_example(example):
   return {
       "CityDescription": example["City Description"],
       "FeatureExtraction": example["Feature Extraction"],
   }
def filter_samples(example):
   CityDescription = []
   FeatureExtraction = []
   for desc, feat in zip(example["City Description"], example["Feature Extraction"]):
       if len(desc) > 0:
           CityDescription.append(desc)
           FeatureExtraction.append(feat)
   return {"CityDescription": CityDescription, "FeatureExtraction": FeatureExtraction}

# Apply transformations to the dataset
geographical_data = geographical_data.map(flatten_example)
geographical_data = geographical_data.map(filter_samples, batched=True)
# Split the dataset into train and validation sets
train_data_txt, validation_data_txt = geographical_data.train_test_split(test_size=0.1).values()


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

#Tokenization and Preprocessing for Sequence-to-Sequence Models
#Prepare the data for training a Seq2Seq model, where the input is the tokenized and processed "CityDescription," and the target is the tokenized and processed "FeatureExtraction."

In [9]:
def preprocess_batch(batch, tokenizer, max_source_length, max_target_length):
   src, tgt = batch["CityDescription"], batch["FeatureExtraction"]
   src_tokenized = tokenizer(
       src, padding="max_length", truncation=True, max_length=max_source_length
   )
   tgt_tokenized = tokenizer(
       tgt, padding="max_length", truncation=True, max_length=max_target_length
   )
   batch = {k: v for k, v in src_tokenized.items()}
   # Ignore padding in the loss
   batch["labels"] = [
       [-100 if token == tokenizer.pad_token_id else token for token in l]
       for l in tgt_tokenized["input_ids"]
   ]
   return batch


#Preprocess the training data for a sequence-to-sequence model.

In [10]:
train_data = train_data_txt.map(
   lambda batch: preprocess_batch(
       batch, tokenizer, encoder_max_len, decoder_max_len
   ),
   batched=True,
   remove_columns=train_data_txt.column_names,
)


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

#Preprocess the validation data suitable for evaluating the performance of a sequence-to-sequence model

In [11]:
validation_data = validation_data_txt.map(
   lambda batch: preprocess_batch(
       batch, tokenizer, encoder_max_len, decoder_max_len
   ),
   batched=True,
   remove_columns=validation_data_txt.column_names,
)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

#Build the Model for fine-tuning.
#Set up the necessary libraries for training and evaluating Seq2Seq models

In [12]:
import numpy as np
from datasets import load_metric
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
nltk.download("punkt", quiet=True)
metric = load_metric("rouge") #Recall-Oriented Understudy for Gisting Evaluation
def postprocess_text(preds, labels):
   preds = [pred.strip() for pred in preds]
   labels = [label.strip() for label in labels]
   # rougeLSum expects newline after each sentence
   preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
   labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
   return preds, labels


<ipython-input-12-13c42cffecff>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge") #Recall-Oriented Understudy for Gisting Evaluation


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


#Define function to evaluate the performance of the model on generated summaries compared to the reference summaries using the ROUGE metric.

In [13]:
def calculate_metrics(eval_preds):
   preds, labels = eval_preds
   if isinstance(preds, tuple):
       preds = preds[0]
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   # Replace -100 in the labels as we can't decode them.
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
   # Some simple post-processing
   decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
   result = metric.compute(
       predictions=decoded_preds, references=decoded_labels, use_stemmer=True
   )
   # Extract a few results from ROUGE
   result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

   prediction_lens = [
       np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
   ]
   result["gen_len"] = np.mean(prediction_lens)
   result = {k: round(v, 4) for k, v in result.items()}
   return result


#Train the model using the provided configuration and datasets.
#Define a Seq2SeqTrainingArguments object that encapsulates the training arguments and configuration.

#Create a DataCollatorForSeq2Seq object that is responsible for collating and processing the training data.

#Build  Seq2SeqTrainer object, to handle training loop, optimization, logging, and evaluation, and manages the training process.

In [14]:
training_args = Seq2SeqTrainingArguments(
   output_dir="results",
   num_train_epochs=1,  # demo
   do_train=True,
   do_eval=True,
   per_device_train_batch_size=4,  # demo
   per_device_eval_batch_size=4,
   # learning_rate=3e-05,
   warmup_steps=500,
   weight_decay=0.1,
   label_smoothing_factor=0.1,
   predict_with_generate=True,
   logging_dir="logs",
   logging_steps=50,
   save_total_limit=3,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   data_collator=data_collator,
   train_dataset=train_data,
   eval_dataset=validation_data,
   tokenizer=tokenizer,
   compute_metrics=calculate_metrics,
)


#Evaluate pre-trained model before training

In [15]:
trainer.evaluate()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: bammidi-abhinav (bammidi-abhinav-christ-university). Use `wandb login --relogin` to force relogin


{'eval_loss': 5.641207218170166,
 'eval_rouge1': 4.8342,
 'eval_rouge2': 0.6667,
 'eval_rougeL': 4.7884,
 'eval_rougeLsum': 4.8211,
 'eval_gen_len': 17.95,
 'eval_runtime': 5.4925,
 'eval_samples_per_second': 3.641,
 'eval_steps_per_second': 0.91}

#Train the model for given dataset

In [16]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=45, training_loss=4.712012736002604, metrics={'train_runtime': 19.312, 'train_samples_per_second': 9.321, 'train_steps_per_second': 2.33, 'total_flos': 55724212224000.0, 'train_loss': 4.712012736002604, 'epoch': 1.0})

#Evaluate the model after training

In [17]:
trainer.evaluate()


{'eval_loss': 3.312502384185791,
 'eval_rouge1': 34.5832,
 'eval_rouge2': 22.3499,
 'eval_rougeL': 34.1657,
 'eval_rougeLsum': 33.8762,
 'eval_gen_len': 16.75,
 'eval_runtime': 2.9869,
 'eval_samples_per_second': 6.696,
 'eval_steps_per_second': 1.674,
 'epoch': 1.0}

In [18]:
def generate_summary(test_data, model):
   inputs = tokenizer(
       test_data["CityDescription"],
       padding="max_length",
       truncation=True,
       max_length=encoder_max_len,
       return_tensors="pt",
   )
   input_ids = inputs.input_ids.to(model.device)
   attention_mask = inputs.attention_mask.to(model.device)
   outputs = model.generate(input_ids, attention_mask=attention_mask)
   output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
   return outputs, output_str
model_before_fine_tuning = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
test_samples = validation_data_txt.select(range(10))
features_before_fine_tuning = generate_summary(test_samples, model_before_fine_tuning)[1]
features_after_fine_tuning = generate_summary(test_samples, model)[1]


In [19]:
print("\nTarget Feature:\n")
print(
   tabulate(list(enumerate(test_samples["FeatureExtraction"])), headers=["Id", "Target Feature"])
)
print("\nCity Description:\n")
print(tabulate(list(enumerate(test_samples["CityDescription"])), headers=["Id", "City Description"]))



Target Feature:

  Id  Target Feature
----  --------------------------------------------------------------------------
   0  Rich history and cultural heritage, Pink City architecture.
   1  Scenic beauty, adventure activities.
   2  Rich history, Sariska Tiger Reserve.
   3  Historical and cultural significance, rapid urbanization.
   4  Religious and cultural significance, agricultural importance.
   5  Well-planned infrastructure, recognized as one of India's greenest cities.
   6  agricultural significance, annual agricultural fair
   7  Historical and religious significance, educational institutions
   8  Textile industry, agricultural activities
   9  Beautiful beaches, historical forts.

City Description:

  Id  City Description
----  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics for evaluating automatic summarization of texts as well as machine translations. It works by comparing an automatically produced summary or translation against a set of reference summaries (typically human-produced). The metrics compare the generated summaries to reference (gold-standard) summaries based on overlapping n-grams (sequences of n consecutive words) and other measures.eval_rouge1 denotes the ROUGE-1 F1-score, eval_rouge2 represents the ROUGE-2 F1-score, eval_rougeL corresponds to the ROUGE-L F1-score, and eval_rougeLsum indicates the ROUGE-Lsum F1-score.
#These scores are used to measure the performance of the summarization model, with higher scores indicating better alignment between the generated summaries and the reference summaries.